In [39]:
import os
import llama_index

In [40]:
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [ ]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader('../src/data')
docs

In [43]:
docs=docs.load_data()
docs

[Document(id_='7392c46d-5b4d-436e-a7c1-b3283f21c096', embedding=None, metadata={'page_label': '1', 'file_name': 'transformer.pdf', 'file_path': 'f:\\Llama_index_Project\\Study_Material\\..\\Study_Material\\data\\transformer.pdf', 'file_type': 'application/pdf', 'file_size': 4445724, 'creation_date': '2025-05-10', 'last_modified_date': '2025-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" Breakdown of the Imports:\n1. import torch – This imports PyTorch, which we’ll use for tensor computations, autograd, and \nmodel building.\n2. import torch.nn as nn – This brings in PyTorch’s neural network module (nn), which contains \nlayers l

In [44]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model

HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002B81967AAE0>, num_workers=None, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None, show_progress_bar=False)

In [45]:
embeddings = Settings.embed_model.get_text_embedding("It is raining cats and dogs here!")
print(len(embeddings))
print(embeddings)

384
[0.017773352563381195, -0.026311516761779785, -0.010715308599174023, 0.09249845892190933, 0.059014905244112015, -0.05219223350286484, 0.03183029964566231, 0.0315842404961586, 0.018293585628271103, 0.010523504577577114, -0.07242640852928162, -0.07518339902162552, 0.03899151086807251, 0.007030792068690062, 0.005894538946449757, -0.015238869935274124, 0.04990237206220627, -0.013445298187434673, -0.08619800209999084, 0.06485205888748169, -0.058302734047174454, 0.014859593473374844, -0.026552777737379074, -0.037582751363515854, -0.04921269416809082, 0.056993518024683, -0.021751316264271736, 0.0634465292096138, -0.03945092484354973, -0.05733480677008629, -0.02479013428092003, -0.017419954761862755, 0.0021395559888333082, 0.015224208123981953, 0.040672678500413895, -0.0038720553275197744, 0.02177206426858902, -0.0313495509326458, 0.0022080063354223967, 0.03819335624575615, 0.004114842973649502, -0.00786332506686449, -0.0929982140660286, -0.01432387251406908, -0.02459550090134144, -0.00957

In [46]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs,show_progress=True)
index


Generating embeddings: 100%|██████████| 30/30 [00:20<00:00,  1.46it/s]


In [ ]:
index.storage_context.persist(persist_dir="../src/vdb")

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="../src/vdb")
storage_context

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000002B8206628D0>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000002B820126420>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={'7640c971-d974-4a3c-a75b-072b5008ec05': [-0.06349977850914001, 0.031074456870555878, -0.0198642797768116, -0.017937615513801575, 0.026424182578921318, 0.01931745558977127, -0.045697666704654694, -0.036361388862133026, -0.02772308513522148, 0.004448611754924059, 0.00954881776124239, -0.044176775962114334, 0.022101635113358498, 0.049437735229730606, -0.011174279265105724, 0.008375357836484909, -0.04411604255437851, -0.01660342887043953, -0.05550959333777428, -0.059856511652469635, 0.03760366886854172, -0.04093070328235626, 0.030248619616031647, -0.03308464214205742, 0.0397665910422802, 0.012341940775513649, -0.0244570877

In [49]:
# load the index from storage
index = load_index_from_storage(storage_context)
index

In [50]:
from llama_index.core.retrievers import VectorIndexRetriever

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
retriever

In [33]:
from llama_index.llms.groq import Groq

Settings.llm = Groq(model="llama3-70b-8192")
Settings.llm

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002B81967AAE0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000002B8001FCB80>, completion_to_prompt=<function default_completion_to_prompt at 0x000002B80081F420>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama3-70b-8192', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, reasoning_effort=None, modalities=None, audio_config=None, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [34]:
response=Settings.llm.complete("Hello World!")
print(response)

The classic greeting! Hello World, indeed! It's great to have you on this platform. Is there something on your mind that you'd like to chat about, or are you just looking to explore and see what's out there? I'm here to help with any questions or topics you'd like to discuss!


In [35]:
from llama_index.core import get_response_synthesizer

# configure response synthesizer
response_synthesizer = get_response_synthesizer(llm=Settings.llm)
response_synthesizer

In [36]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever=retriever,response_synthesizer=response_synthesizer)
query_engine

In [53]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]


resp = Settings.llm.chat(messages)
print(resp)

assistant: Arrr, me hearty! Me name be Captain Zephyr "Blackheart" McSnively, the most feared and revered pirate to ever sail the seven seas! Me reputation precedes me, and me exploits be the stuff o' legend! Me ship, the "Maverick's Revenge", be me home, me pride, and me ticket to adventure and riches!

Now, what be bringin' ye to these fair waters? Are ye lookin' to join me crew, or are ye lookin' to cross swords with the greatest pirate that ever lived? Either way, ye be in fer a wild ride!


In [54]:
response = Settings.llm.stream_complete("Principle of Transformer")
for r in response:
    print(r.delta, end="")

The principle of a transformer is based on the concept of electromagnetic induction, which states that a changing magnetic field induces an electromotive force (EMF) in a conductor. The transformer works on the principle of mutual induction, where two coils are placed close to each other, and the magnetic field generated by one coil induces an EMF in the other coil.

Here's a step-by-step explanation of the principle of a transformer:

1. **Primary Coil**: The primary coil is connected to an AC power source, which causes an alternating current (AC) to flow through it. This creates a magnetic field around the primary coil.
2. **Magnetic Field**: The magnetic field generated by the primary coil induces an electromotive force (EMF) in the secondary coil.
3. **Secondary Coil**: The secondary coil is placed close to the primary coil, and the magnetic field generated by the primary coil induces an EMF in the secondary coil.
4. **Induction**: The EMF induced in the secondary coil is proportio

In [38]:
# query
response = query_engine.query("Principle of Transformer")
print(response)

The principle of the Transformer is based on Self-Attention, which enables the model to focus on relevant tokens in the target sequence while ensuring that future tokens aren't attended to. Additionally, Cross-Attention allows the decoder to relevant parts of the source sequence, and the Feedforward Layer refines the representations of each token independently.


In [17]:
queries = [
    "What is a transformer model?",
    "Explain the positional encoding in transformers",
    "How does multi-head attention work?",
    "What are the special tokens used in transformer models?",
    "How is the vocabulary created for transformer models?"
]